<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/gn/MOM_19th_April.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y nltk
!pip install --no-cache-dir nltk==3.8.1

import nltk
nltk.download("punkt")

from nltk import word_tokenize
tokens = word_tokenize("This should now work.")
print("Tokenized:", tokens)



Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 99.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.
Tokenized: ['This', 'should', 'now', 'work', '.']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install all the dependencies. Place any new installables into requirements.txt file.
!pip install -r /content/drive/MyDrive/requirements.txt

In [ ]:

import time
import nltk
import torch
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import plotly.graph_objects as go
!pip install fpdf
from fpdf import FPDF
from io import BytesIO
from PIL import Image
import plotly.io as pio

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

from transformers import pipeline
import torch

transcription_pipelines = {
    "whisper-tiny.en": pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-tiny.en",
        device=0 if torch.cuda.is_available() else -1
    ),
    "wav2vec2-bert": pipeline(
        "automatic-speech-recognition",
        model="hf-audio/wav2vec2-bert-CV16-en",
        device=0 if torch.cuda.is_available() else -1
    ),
    "openai/whisper-base.en": pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-base.en",
        device=0 if torch.cuda.is_available() else -1
    )
}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
summarizer_pipelines = {
    "facebook/bart-large-cnn": pipeline("summarization", model="facebook/bart-large-cnn"),
    "philschmid/bart-large-cnn-samsum": pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
    # "google/pegasus-xsum": pipeline("summarization", model="google/pegasus-xsum"),
    # "facebook/bart-large-xsum": pipeline("summarization", model="facebook/bart-large-xsum"),
    # "Falconsai/text_summarization": pipeline("summarization", model="Falconsai/text_summarization")
}


Device set to use cpu
Device set to use cpu


In [ ]:
pip install -U kaleido

In [ ]:
def transcribe_audio(file, model_name):

    pipe = transcription_pipelines.get(model_name)

    if pipe is None:
        raise ValueError("Unsupported model")

    start = time.time()


    if "whisper" in model_name:
        result = pipe(file, return_timestamps=True)
    else:
        result = pipe(file)

    end = time.time()
    return result["text"], f"Transcription time: {end - start:.2f} seconds"


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import textwrap
import time
import gc

def summarize_text(text, model_name):

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    device = 0 if torch.cuda.is_available() else -1
    device_str = "cuda" if device == 0 else "cpu"
    print(f"Device set to use: {device_str}")

    try:

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cpu")


        summarizer = pipeline(
            "summarization",
            model=model,
            tokenizer=tokenizer,
            device=-1
        )
    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        return "[Error loading model]", 0.0


    max_chunk_size = 900
    chunks = textwrap.wrap(text, max_chunk_size)

    final_summary = ""
    start_time = time.time()

    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)} with {model_name}")
        try:
            if chunk.strip() and len(chunk.strip().split()) > 5:
                output = summarizer(
                    chunk,
                    max_length=130,
                    min_length=30,
                    do_sample=False
                )
                summary_text = output[0]['summary_text'].strip()

                if "CNN.com will feature" not in summary_text:
                    final_summary += summary_text + " "
                else:
                    print("Detected fallback summary, skipping.")
            else:
                print("Skipping empty or short chunk.")
        except Exception as e:
            print(f"Error summarizing chunk with {model_name}: {e}")
            final_summary += "[Error summarizing chunk] "

    end_time = time.time()
    elapsed_time = round(end_time - start_time, 2)

    return final_summary.strip(), elapsed_time


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.feature_extraction.text import CountVectorizer

def evaluate_summary(reference_text, summary):
    nltk.download('punkt', quiet=True)


    ref_tokens = nltk.word_tokenize(reference_text.lower())
    pred_tokens = nltk.word_tokenize(summary.lower())


    bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=SmoothingFunction().method1)


    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference_text, summary)


    vectorizer = CountVectorizer(binary=True)
    vectors = vectorizer.fit_transform([reference_text, summary]).toarray()
    y_true, y_pred = vectors[0], vectors[1]


    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)

    return {
        "BLEU": round(bleu, 2),
        "ROUGE1": round(rouge_scores["rouge1"].fmeasure, 2),
        "ROUGEL": round(rouge_scores["rougeL"].fmeasure, 2),
        "PRECISION": round(precision, 2),
        "RECALL": round(recall, 2),
        "F1": round(f1, 2),
        "ACCURACY": round(accuracy, 2),
    }


In [ ]:
def show_loader(text):
    return gr.update(value=f" {text}...")

In [ ]:
def format_action_items(action_items_by_model):
    formatted = "<h4>Action Items</h4>"
    for model, items in action_items_by_model.items():
        formatted += f"<b>{model}:</b><br>"
        if items:
            formatted += "<ul>" + "".join(f"<li>{item}</li>" for item in items) + "</ul>"
        else:
            formatted += "<i>No action items found.</i><br>"
    return formatted


In [ ]:
def extract_action_items(summary, keywords=None):
    if not keywords:
        keywords = ["must", "should", "need to", "expanding", "revenue", "plan", "action"]
    keywords = [kw.lower() for kw in keywords]
    sentences = summary.split(". ")
    return [s for s in sentences if any(kw in s.lower() for kw in keywords)]

In [ ]:
from fpdf import FPDF
import os
import datetime
def generate_pdf(transcription, results, graph_paths):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "Minutes of Meeting Report", ln=True)


    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Transcribed Text", ln=True)
    pdf.set_font("Arial", size=11)
    pdf.multi_cell(0, 10, transcription)


    for model, data in results.items():
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 10, f"Summary - {model}", ln=True)
        pdf.set_font("Arial", size=11)
        pdf.multi_cell(0, 10, data['summary'])

        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, "Action Items", ln=True)
        pdf.set_font("Arial", size=11)
        for item in data['actions']:
            pdf.cell(5)
            pdf.multi_cell(0, 10, f"- {item}")

        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, "Evaluation Metrics", ln=True)
        pdf.set_font("Arial", size=11)
        for k, v in data["metrics"].items():
            pdf.cell(0, 10, f"{k.upper()}: {v:.2f}", ln=True)


    titles = [
        "Bar Chart - Evaluation Metrics",
        f"Pie Chart - {list(results.keys())[0]}",
        f"Pie Chart - {list(results.keys())[1]}" if len(results) > 1 else "Pie Chart",
        "Line Chart - Evaluation Metrics",
        "Radar Chart - Evaluation Metrics"
    ]

    for idx, path in enumerate(graph_paths):
        if idx % 2 == 0:
            pdf.add_page()

        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, titles[idx], ln=True)
        pdf.image(path, w=180)
        pdf.ln(5)

    filename = f"MoM_Report_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    file_path = os.path.join(os.getcwd(), filename)
    pdf.output(file_path)
    return file_path


In [ ]:
def show_graphs(results):
    if not results or len(results) < 2:
        return (gr.update(visible=False),) * 4


    model_names = list(results.keys())
    metrics = list(next(iter(results.values()))["metrics"].keys())


    fig1 = go.Figure()
    for model in model_names:
        fig1.add_trace(go.Bar(name=model,
                              x=metrics,
                              y=[results[model]["metrics"].get(m, 0.0) for m in metrics]))
    fig1.update_layout(
        title="Evaluation Metrics - Bar Chart",
        barmode='group',
        xaxis_title="Metrics",
        yaxis_title="Score",
        yaxis=dict(range=[0, 1])
    )


    fig2 = go.Figure()
    for model in model_names:
        fig2.add_trace(go.Pie(
            labels=metrics,
            values=[results[model]["metrics"].get(m, 0.0) for m in metrics],
            name=model,
            hole=0.3,
            title=model,
            domain=dict(row=0, column=model_names.index(model))
        ))
    fig2.update_layout(title="Evaluation Metrics - Pie Chart", grid=dict(rows=1, columns=len(model_names)))


    fig3 = go.Figure()
    for model in model_names:
        fig3.add_trace(go.Scatter(
            x=metrics,
            y=[results[model]["metrics"].get(m, 0.0) for m in metrics],
            mode="lines+markers",
            name=model
        ))
    fig3.update_layout(title="Evaluation Metrics - Line Chart", yaxis=dict(range=[0, 1]))


    fig4 = go.Figure()
    for model in model_names:
        values = [results[model]["metrics"].get(m, 0.0) for m in metrics]
        values += values[:1]
        fig4.add_trace(go.Scatterpolar(
            r=values,
            theta=metrics + [metrics[0]],
            fill='toself',
            name=model
        ))
    fig4.update_layout(
        title="Evaluation Metrics - Radar Chart",
        polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
        showlegend=True
    )

    return (
        gr.update(value=fig1, visible=True),
        gr.update(value=fig2, visible=True),
        gr.update(value=fig3, visible=True),
        gr.update(value=fig4, visible=True)
    )


In [ ]:
def show_graphs1(results):
    import plotly.graph_objects as go

    models = list(results.keys())
    metrics = list(results[models[0]]["metrics"].keys())

    fig_bar = go.Figure()
    fig_line = go.Figure()
    for model in models:
        values = list(results[model]["metrics"].values())
        fig_bar.add_trace(go.Bar(name=model, x=metrics, y=values))
        fig_line.add_trace(go.Scatter(name=model, x=metrics, y=values, mode='lines+markers'))

    fig_bar.update_layout(title="Evaluation Metrics - Bar Chart", barmode='group')
    fig_line.update_layout(title="Evaluation Metrics - Line Chart")


    fig_radar = go.Figure()
    for model in models:
        values = list(results[model]["metrics"].values())
        values += values[:1]
        fig_radar.add_trace(go.Scatterpolar(r=values, theta=metrics + [metrics[0]], fill='toself', name=model))
    fig_radar.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), title="Radar Chart")


    fig_pie1 = go.Figure(go.Pie(
        labels=metrics,
        values=list(results[models[0]]["metrics"].values()),
        title=f"Pie Chart - {models[0]}"
    ))

    fig_pie2 = go.Figure(go.Pie(
        labels=metrics,
        values=list(results[models[1]]["metrics"].values()),
        title=f"Pie Chart - {models[1]}"
    ))

    return fig_bar, fig_pie1, fig_pie2, fig_line, fig_radar


In [ ]:
def process_and_format_outputs(file, trans_model, keywords, summarizer_models):
    try:
        print("=== PROCESS STARTED ===")
        print(f"Selected Transcription Model: {trans_model}")
        print(f"Selected Summarizer Models: {summarizer_models}")


        transcription, trans_time = transcribe_audio(file, trans_model)
        print("Transcribed")
        yield transcription, "Generating summary...", "Generating summary...", f"{trans_time}", "", "", {}, None, "Transcription complete."


        results = {}
        from concurrent.futures import ThreadPoolExecutor
        with ThreadPoolExecutor() as executor:
            futures = {
                model: executor.submit(summarize_text, transcription, model)
                for model in summarizer_models
            }
            for model, future in futures.items():
                print(f"Summarizing with: {model}")
                summary, summ_time_raw = future.result()


                try:
                    summ_time = float(summ_time_raw)
                except:
                    summ_time = 0.0

                print(f"Summary complete for: {model}")
                actions = extract_action_items(summary, keywords)
                metrics = evaluate_summary(transcription, summary)
                results[model] = {
                    "summary": summary,
                    "actions": actions,
                    "metrics": metrics,
                    "times": {
                        "transcription": trans_time,
                        "summarization": summ_time
                    }
                }


        summary1 = results.get(summarizer_models[0], {}).get("summary", "")
        summary2 = results.get(summarizer_models[1], {}).get("summary", "") if len(summarizer_models) > 1 else ""

        formatted_actions = format_action_items({m: r["actions"] for m, r in results.items()})


        metric_keys = list(next(iter(results.values()))["metrics"].keys())
        metrics_html = "<h4>Evaluation Metrics</h4><table border='1' cellpadding='6' style='border-collapse: collapse;'>"
        metrics_html += "<tr><th>Metric</th>" + "".join(f"<th>{model}</th>" for model in results.keys()) + "</tr>"
        for metric in metric_keys:
            metrics_html += "<tr>"
            metrics_html += f"<td><b>{metric}</b></td>"
            for model in results.keys():
                val = results[model]["metrics"].get(metric, 0.0)
                metrics_html += f"<td>{val:.2f}</td>"
            metrics_html += "</tr>"
        metrics_html += "</table>"


        fig_bar, fig_pie1, fig_pie2, fig_line, fig_radar = show_graphs1(results)
        graph_paths = []
        for i, fig in enumerate([fig_bar, fig_pie1, fig_pie2, fig_line, fig_radar], start=1):
            path = f"graph_{i}.png"
            fig.write_image(path)
            graph_paths.append(path)


        summarization_times = "\n".join([
            f"Summarization time ({model}): {results[model]['times']['summarization']:.2f} seconds"
            for model in results
        ])
        time_output = f"{trans_time}\n{summarization_times}"


        pdf_filename = generate_pdf(
            transcription=transcription,
            results=results,
            graph_paths=graph_paths
        )

        yield transcription, summary1, summary2, time_output, formatted_actions, metrics_html, results, pdf_filename, "Summarization complete."

    except Exception as e:
        error_msg = f"Error during processing: {str(e)}"
        print("EXCEPTION:", error_msg)
        yield error_msg, "", "", "", "", "", {}, None, "Error"


In [ ]:
!pip install gradio
import gradio as gr
from transformers import pipeline
import torch
import time
import re
import plotly.graph_objects as go
from gradio.themes.soft import Soft
from gradio.themes.utils import colors

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('wordnet')


custom_theme = Soft(primary_hue=colors.blue, font=["Inter", "sans-serif"])


def generate_comparison_table(eval_data):
    html = """
    <h4>Comparison Table</h4>
    <table border='1' style='border-collapse:collapse; width:100%; text-align:left;'>
        <tr>
            <th style='width:20%;'>Model</th>
            <th style='width:60%;'>Summary</th>
            <th style='width:20%;'>Metrics</th>
        </tr>
    """
    for model, data in eval_data.items():
        summary = data.get('summary', '').replace('\n', '<br>')
        metrics = data.get('metrics', {})
        metric_html = "<br>".join([f"{k.upper()}: {v:.2f}" for k, v in metrics.items()])

        html += f"""
        <tr>
            <td style='width:20%; vertical-align:top;'>{model}</td>
            <td style='width:60%; vertical-align:top;'>{summary}</td>
            <td style='width:20%; vertical-align:top;'>{metric_html}</td>
        </tr>
        """
    html += "</table>"
    return html

def update_summary_labels(selected_models):
    label1 = f"Summary - {selected_models[0]}" if len(selected_models) > 0 else "Summary - Model 1"
    label2 = f"Summary - {selected_models[1]}" if len(selected_models) > 1 else "Summary - Model 2"
    visible2 = len(selected_models) > 1
    return (
        gr.Textbox.update(label=label1),
        gr.Textbox.update(label=label2, visible=visible2)
    )

def live_audio(audio_type):
    transcribe_audio = gr.Interface(
                transcribe_audio,
                gr.Audio(sources="microphone"),
                "text",
            )
    return transcribe_audio

with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("### Minutes of Meeting")

    evaluation_state = gr.State()
    summary_box_labels = gr.State({})

    with gr.Row():
        with gr.Column(scale=1):

            audio_mode = gr.Radio(
                choices=["Live recording", "Upload audio"],
                value="Upload audio",
                label="Select Audio Input Mode"
            )


            live_audio_input = gr.Audio(type="filepath", label="Record your audio", visible=False, interactive=True, show_label=True)

            file_input = gr.File(label="Upload your audio file", visible=True)

            def toggle_audio_inputs(mode):
                return (
                    gr.update(visible=(mode == "Live recording")),
                    gr.update(visible=(mode == "Upload audio"))
                )

            audio_mode.change(
                fn=toggle_audio_inputs,
                inputs=audio_mode,
                outputs=[live_audio_input, file_input]
            )

            with gr.Accordion("Advanced Settings", open=False):
                trans_dropdown = gr.Dropdown(
                    choices=["whisper-tiny.en", "wav2vec2-bert", "openai/whisper-base.en"],
                    label="Select Transcription Model",
                    value="whisper-tiny.en",
                )

                summarizer_models = gr.CheckboxGroup(
                    choices=[
                        "facebook/bart-large-cnn",
                        "philschmid/bart-large-cnn-samsum",
                        "google/pegasus-xsum",
                        "facebook/bart-large-xsum",
                        "Falconsai/text_summarization"
                    ],
                    label="Select Summarizer Models (max 2)",
                    value=[
                        "facebook/bart-large-cnn",
                        "philschmid/bart-large-cnn-samsum"
                    ],
                    interactive=True
                )

                def limit_two_models(selection):
                    return selection[:2] if len(selection) > 2 else selection

                summarizer_models.change(
                    fn=limit_two_models,
                    inputs=summarizer_models,
                    outputs=summarizer_models
                )

                keyword_input = gr.Textbox(
                    label="Custom Keywords (comma-separated)",
                    placeholder="e.g. must, should, revenue, action"
                )

            btn_all = gr.Button("Process All")

            with gr.Accordion("Evaluation Metric Graphs", open=False):
                with gr.Tabs():
                    with gr.TabItem("Bar Chart"):
                        graph1 = gr.Plot(visible=False)
                    with gr.TabItem("Pie Chart"):
                        graph2 = gr.Plot(visible=False)
                    with gr.TabItem("Line Chart"):
                        graph3 = gr.Plot(visible=False)
                    with gr.TabItem("Radar Chart"):
                        graph4 = gr.Plot(visible=False)

            btn_show_graphs = gr.Button("Show Evaluation Metrics")
            btn_show_graphs.click(
                fn=show_graphs,
                inputs=[evaluation_state],
                outputs=[graph1, graph2, graph3, graph4]
            )

            comparison_table = gr.HTML(label="Summary & Metric Comparison Table")
            btn_compare = gr.Button("Show Comparison Table")
            btn_compare.click(fn=generate_comparison_table, inputs=[evaluation_state], outputs=comparison_table)

            output_plot = gr.HTML(label="Evaluation Plot (Static)")
            output_download = gr.File(label="Download Report")

        with gr.Column(scale=1):
            progress_box = gr.Textbox(label="Progress", lines=2, interactive=False)
            summary_boxes = gr.State({})

            output_trans = gr.Textbox(label="Transcribed Text", lines=10)
            output_summ1 = gr.Textbox(label="Summary - Model 1", lines=6, interactive=False)
            output_summ2 = gr.Textbox(label="Summary - Model 2", lines=6, interactive=False, visible=True)
            output_time = gr.Textbox(label="Processing Time", lines=2)
            output_action = gr.HTML(label="Action Items")
            output_metrics = gr.HTML(label="Evaluation Metrics")

    def set_summary_visibility(selected_models):
        visible = len(selected_models) > 1
        return gr.Textbox.update(visible=visible)

    summarizer_models.change(
        fn=set_summary_visibility,
        inputs=summarizer_models,
        outputs=output_summ2
    )

    def route_audio(live_audio_path, file_audio_path, audio_mode, model, keywords, summarizers):

        if "Live recording" in audio_mode and live_audio_path is not None:
            file = live_audio_path
        elif "Upload audio" in audio_mode and file_audio_path is not None:
            file = file_audio_path
        else:
            yield "Please provide an audio input.", "", "", "", "", "", {}, None, "Error"
            return

        yield from process_and_format_outputs(file, model, keywords, summarizers)


    btn_all.click(
        fn=route_audio,
        inputs=[live_audio_input, file_input, audio_mode, trans_dropdown, keyword_input, summarizer_models],
        outputs=[
            output_trans,
            output_summ1,
            output_summ2,
            output_time,
            output_action,
            output_metrics,
            evaluation_state,
            output_download,
            progress_box
        ],
        queue=True
    )

demo.queue().launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://641f283ab3b5fa6490.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


=== PROCESS STARTED ===
Selected Transcription Model: whisper-tiny.en
Selected Summarizer Models: ['facebook/bart-large-cnn', 'philschmid/bart-large-cnn-samsum']


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcribed
Device set to use: cpu
Summarizing with: facebook/bart-large-cnn
Device set to use: cpu


Device set to use cpu


...

Error loading model facebook/bart-large-cnn: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.
Summary complete for: facebook/bart-large-cnn
Summarizing with: philschmid/bart-large-cnn-samsum
Summary complete for: philschmid/bart-large-cnn-samsum
=== PROCESS STARTED ===
Selected Transcription Model: whisper-tiny.en
Selected Summarizer Models: ['facebook/bart-large-cnn', 'philschmid/bart-large-cnn-samsum']


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning:

The input name `inputs` is deprecated. Please make sure to use `input_features` instead.

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Transcribed
Device set to use: cpu
Device set to use: cpu
Summarizing with: facebook/bart-large-cnn


Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['lm_head.weight', 'model.encoder.embed_tokens.weight', 'model.shared.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error loading model facebook/bart-large-cnn: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.
Summary complete for: facebook/bart-large-cnn
Summarizing with: philschmid/bart-large-cnn-samsum
Error loading model philschmid/bart-large-cnn-samsum: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.
Summary complete for: philschmid/bart-large-cnn-samsum
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://641f283ab3b5fa6490.gradio.live
